In [ ]:
import pandas as pd
#1.确定这几个数据都是干什么的
swiping_card_dataframe = pd.read_csv('./bus_swiping_card_info1.csv')
card_info_dataframe = pd.read_csv('./card_info.csv')
bus_swip_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
bus_station_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')
#swiping_card_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间
#card_info_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间也是上面的信息，但是时间是去年的。
#用去年的时间的同比数据，去预测今年的同比数据，看是否一致，如果一致，说明经济发展没有变化，如果变大，说明经济好转
#bus_swip_dataframe ：一天的统计信息：各条线路、各个站点的刷卡数量信息
#bus_station_dataframe ： 站点的统计信息：各个站点的经纬度
station_lat_lon_dataframe = bus_station_dataframe.groupby('station_name').agg({'latitude':list,'longitude':list}).reset_index()
station_lat_lon_dataframe['lat'] = station_lat_lon_dataframe['latitude'].apply(lambda x:x[0])
station_lat_lon_dataframe['lon'] = station_lat_lon_dataframe['longitude'].apply(lambda x:x[0])
station_lat_lon_dataframe.drop(labels=['latitude','longitude'],axis=1,inplace=True)
station_lat_lon_dataframe['lat_lon']=station_lat_lon_dataframe.apply(lambda x:(x['lat'],x['lon']),axis=1)
station_lat_lon_dataframe.drop(labels=['lat','lon'],axis=1,inplace=True)
station_lat_lon_ndarray = station_lat_lon_dataframe.values

In [ ]:
station_lat_lon_ndarray

In [ ]:
station_sequence_ndarray = station_lat_lon_dataframe['station_name'].values
station_sequence_ndarray
import numpy as np
np.savetxt('./station_sequence_ndarray.txt',station_sequence_ndarray,fmt='%s')
station_nums_setting = len(station_sequence_ndarray)

In [ ]:
import itertools
from geopy import distance
iters=0
station_pair_distance=[]
station_pair_distance_temp=[]
for station_pair_instance in itertools.product(station_lat_lon_ndarray,station_lat_lon_ndarray):
    station_0 = station_pair_instance[0]
    station_1 = station_pair_instance[1]
    station_0_position = station_0[1]
    station_1_position = station_1[1]
    dist = distance.distance(station_0_position,station_1_position).m
    if iters % station_nums_setting==0:
        if iters!=0:
            station_pair_distance.append(station_pair_distance_temp)
        station_pair_distance_temp=[]
        station_pair_distance_temp.append(dist)
    else:
        station_pair_distance_temp.append(dist)
    iters +=1
print(station_pair_distance[0])

In [ ]:
import csv
# 保存列表到文件中
with open('./station_pair_distance.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(station_pair_distance)
print("文件保存成功！")

In [ ]:
station_pair_distance_ndarray = np.array(station_pair_distance)
# 找出每行最小的k个值所对应的索引
station_distance_rank_index_list = [sorted(range(len(row)), key=lambda i: row[i]) for row in station_pair_distance_ndarray]
# 保存列表到文件中
with open('./station_pair_distance_rank_index.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(station_distance_rank_index_list)
print("文件保存成功！")

In [ ]:
station_sequence_ndarray = station_lat_lon_dataframe['station_name'].values
station_sequence_ndarray

In [ ]:
# 创建字典并填充数据
station_2_id = {value: index for index, value in enumerate(station_sequence_ndarray)}
station_2_id

In [ ]:
# 创建字典并填充数据
id_2_station = {index: value for index, value in enumerate(station_sequence_ndarray)}
id_2_station

In [ ]:
import json
# 将字典保存到文件中
with open('./id_2_station.json', 'w') as file:
    json.dump(id_2_station, file)
# 将字典保存到文件中
with open('./station_2_id.json', 'w') as file:
    json.dump(station_2_id, file)